#### Author: Madhusudhanan Balasubramanian (MB), Ph.D., The University of Memphis
#### Axon area size and eccentricity distribution in the UTHSC training, validation and testing datasets as well as in the JHU testing dataset
#### Statistics based on mask area reported in the GT files
#### MB 03/27/2025
#### MB 03/30/2025 - v2: Generic binning by area distribution; min area: 0; max area: 1024^2 ~ 1.1e6

In [ ]:
import cv2
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.structures import BoxMode
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict  # Use defaultdict for easier grouping
from math import ceil
import os

def calculate_polygon_area(segmentation):
    """
    Calculate the area of a polygon given its segmentation coordinates.
    :param segmentation: List of coordinates in the format [x1, y1, x2, y2, ...].
    :return: Area of the polygon.
    """
    # Reshape the segmentation into a 2D array of points [[x1, y1], [x2, y2], ...]
    points = np.array(segmentation).reshape(-1, 2)
    #print(f"points: {points}")
    
    # Use the Shoelace formula to calculate the area
    x = points[:, 0]
    y = points[:, 1]
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    #print(f"\t area: {area}")
    return area

#Use the following to get binned stats using fixed bin boundaries
#Usage: binned_stats = get_binned_stats(sizes, size_bins)
def get_binned_stats(sizes, bins):
    """Calculates median, range, and proportion statistics within specified bins."""
    #print(f"bins: {bins}")
    binned_stats = {}
    total_count = len(sizes)
    for i in range(len(bins) - 1):
        bin_min, bin_max = bins[i], bins[i + 1]
        binned_sizes = [s for s in sizes if bin_min <= s < bin_max]
        item_indices = [i for i, s in enumerate(sizes) if bin_min <= s < bin_max]
        if binned_sizes: 
            binned_stats[f"[{bin_min}-{bin_max})"] = {
                "count": len(binned_sizes),
                "proportion": len(binned_sizes) / total_count,
                "median": np.median(binned_sizes),
                "range": (np.min(binned_sizes), np.max(binned_sizes)),
                "item_indices": item_indices
            }
        else:
            binned_stats[f"[{bin_min}-{bin_max})"] = {
                "count": 0,
                "proportion": 0,
                "median": 0,
                "range": (0, 0),
                "item_indices": []
            }
    return binned_stats

#Use the following to get binned stats using quantile distribution of the combined data as well as quantile distribution by category
#Usage: binned_stats = get_binned_stats_v1(sizes, len(size_bins) - 1)
def get_binned_stats_v1(sizes, no_of_levels):
    """Calculates median, range, and proportion statistics within specified bins."""
    binned_stats = {}
    total_count = len(sizes)

    # Calculate quantiles to define bin boundaries
    quantiles = np.linspace(0, 1, no_of_levels + 1)
    bin_boundaries = np.quantile(sizes, quantiles)
    print(f"bin_boundaries: {bin_boundaries}")

    # Assign each value to its corresponding bin
    bins = np.searchsorted(bin_boundaries, sizes, side='right') - 1

    for i in range(no_of_levels):
        bin_min, bin_max = bin_boundaries[i], bin_boundaries[i + 1]
        binned_sizes = [s for j, s in enumerate(sizes) if bins[j] == i]
        if binned_sizes: 
            binned_stats[f"[{bin_min}-{bin_max})"] = {
                "count": len(binned_sizes),
                "proportion": len(binned_sizes) / total_count,
                "median": np.median(binned_sizes),
                "range": (np.min(binned_sizes), np.max(binned_sizes))
            }
        else:
            binned_stats[f"[{bin_min}-{bin_max})"] = {
                "count": 0,
                "proportion": 0,
                "median": 0,
                "range": (0, 0)
            }
    return binned_stats

#Use the following to get binned stats using quantile distribution of the combined data
#Usage: binned_stats, bin_boundaries = get_binned_stats_v2(sizes_all, len(size_bins) - 1)
def get_binned_stats_v2(sizes, no_of_levels):
    """Calculates median, range, and proportion statistics within specified bins."""
    binned_stats = {}
    total_count = len(sizes)

    # Calculate quantiles to define bin boundaries
    quantiles = np.linspace(0, 1, no_of_levels + 1)
    bin_boundaries = np.quantile(sizes, quantiles)
    #print(f"bin_boundaries: {bin_boundaries}; type(bin_boundaries): {type(bin_boundaries)}")

    binned_stats = get_binned_stats(sizes, bin_boundaries)

    return binned_stats, bin_boundaries

def analyze_object_attributes(dataset_name, json_file, image_root):
    """Calculates and visualizes the distribution, median, and range by category of instance characteristics

    Args:
        dataset_name (str): Name of the registered dataset in Detectron2.
    """
    MIN_SIZE_TEST = 1024.0 #all images scaled to this size during testing
    #original_image_sz = 256.0
    #size_factor = MIN_SIZE_TEST / original_image_sz
    scale_flag = True #scale the eccentricity and area statistics based on the final image size used for training/testing
    #size_bins = [-1, 32, 256, 512, 2048, 1e8]
    #
    #ecc boundaries obtained from detailed investigation of model performance in 
    # selecting sizes of interest parameter for Centermask2; see notes for details
    bin_bounds_by_category = {}
    bin_bounds_by_category[0] = [-1, 23, 28, 1024]
    bin_bounds_by_category[1] = [-1, 23, 28, 1024]
    #area_bin_boundaries obtained by running "get_binned_stats_v2(area_all, 3)" on 
    #Phase3_Training dataset
    #area_bin_boundaries = [24,1304, 2256, 35288]
    area_bin_boundaries = [0,1304, 2256, 1.1e6]
    #Load all data/images in the current dataset <dataset_name>
    if dataset_name not in DatasetCatalog.list():  # Check if already registered
        register_coco_instances(dataset_name, {}, json_file, image_root)    
    dataset_dicts = DatasetCatalog.get(dataset_name) #Load annotations of all images in the dataset <dataset_name>
    metadata = MetadataCatalog.get(dataset_name)

    #Initialize
    eccentricity_by_category = defaultdict(list) #creates a dictionary with missing keys initialized with an empty list (because list is specified here)
    eccentricity_all = []
    area_by_category = defaultdict(list)
    area_all = []

    #For all data/image in the dataset, estimate the eccentricity and area attributes of annotations
    for d in dataset_dicts:
        #print(f"image id: {d['image_id']}")
        if scale_flag:
            original_image_sz = min(d["height"], d["width"])
            ecc_factor = MIN_SIZE_TEST / original_image_sz
            area_factor = ecc_factor ** 2
        else:
            ecc_factor = 1
            area_factor = 1

        #process each annotation "ann" in each of the data/image in "d"
        ind = 0
        for ann in d["annotations"]: 
            #print(f"ind: {ind}")
            ind = ind + 1
            category_id = ann["category_id"]
            if not ann.get("iscrowd", 0):  # Skip crowd annotations
                if ann["bbox_mode"] == BoxMode.XYWH_ABS:
                    bbox = ann["bbox"]
                    eccentricity = ceil(max(bbox[2], bbox[3]) / 2) * ecc_factor
                    #
                    eccentricity_by_category[category_id].append(eccentricity)
                    eccentricity_all.append(eccentricity)
                
                    #Estimate area of each of the annotations
                    seg_coords = ann["segmentation"]
                    seg_coords = seg_coords[-1] #In some cases, there are instances within instances (e.g. image 5138, annotation 55)
                    ann_area = calculate_polygon_area(seg_coords) * area_factor
                    area_by_category[category_id].append(ann_area)
                    area_all.append(ann_area)
    
    #To allow indexing operation
    area_all_np = np.array(area_all)
    eccentricity_all_np = np.array(eccentricity_all)

    #Dataset information
    print(f"\n Dataset: {dataset_name}; no of images: {len(dataset_dicts)}")
    
    #Basic ecc and area statistics - overall and by category
    print(f"\n Basic statistics: Overall")
    print_basic_stats(eccentricity_all, "Eccentricity")
    print_basic_stats(area_all, "Area")
    #
    print(f"\n Basic stats: By category")
    for category_id, eccentricity_cat in eccentricity_by_category.items():
        print_basic_stats(eccentricity_cat, f"Cat: {category_id} eccentricity")
        area_cat = area_by_category[category_id]
        print_basic_stats(area_cat, f"Cat: {category_id} area")

    #Generic binned statistics
    print(f"\n Statistics based on genric area binning: Overall")
    #area_binned_stats, bin_boundaries = get_binned_stats_v2(area_all, 3)
    area_binned_stats = get_binned_stats(area_all, area_bin_boundaries)
    print_ecc_area_statistics_v1(area_sel=area_all, eccentricity_all=eccentricity_all_np, area_binned_stats=area_binned_stats)
    #
    print(f"\n Statistics based on generic area binning: By category")
    for category_id, area_sel in area_by_category.items():
        area_binned_stats = get_binned_stats(area_sel, area_bin_boundaries)
        print(f"Cat ID: {category_id}:")
        print_ecc_area_statistics_v1(area_sel=area_sel, eccentricity_all=eccentricity_all_np, area_binned_stats=area_binned_stats)

    #Custom binned statistics
    print(f"\n Statistics based on custom eccentricity binning: Overall")
    ecc_binned_stats = get_binned_stats(eccentricity_all, bin_bounds_by_category[0])
    print_ecc_area_statistics(eccentricity_sel=eccentricity_all, area_all=area_all_np, ecc_binned_stats=ecc_binned_stats)
    #
    print(f"\n Statistics based on custom eccentricity binning: By category")
    for category_id, eccentricity_sel in eccentricity_by_category.items():
        ecc_binned_stats = get_binned_stats(eccentricity_sel, bin_bounds_by_category[category_id])
        print(f"Cat ID: {category_id}:")
        print_ecc_area_statistics(eccentricity_sel=eccentricity_sel, area_all=area_all_np, ecc_binned_stats=ecc_binned_stats)

    #
    #print("\nEccentricity statistics - custom bin sizes:")
    #for category_id, eccentricity in eccentricity_by_category.items():
    #    binned_stats = get_binned_stats(eccentricity, bin_bounds_by_category[category_id])
    #    print(f"Cat ID: {category_id}:")
    #    print_attr_statistics(measure_label="Ecentricity", axon_measure=eccentricity, binned_stats_axon_measure=binned_stats)

def print_ecc_area_statistics_v1(area_sel, eccentricity_all, area_binned_stats):
    """ Prints eccentricity and area statistics.  Assume that ecentricity and 
        area arrays are synchronized with their object IDs.
    """

    total_obj_count = len(eccentricity_all)
    # Print binned statistics for eccentricity if provided
    for bin_range, area_stats in area_binned_stats.items():
        print(f"\t {bin_range}:")
        print(f"\t\t Count: {area_stats['count']}")
        print_binned_stats(area_stats, 'Area')
        #
        sel_indices = area_stats["item_indices"]
        sel_ecc_vec = eccentricity_all[sel_indices]
        ecc_stats = {}
        ecc_stats["count"] = len(sel_ecc_vec)
        ecc_stats["proportion"] = ecc_stats["count"] / total_obj_count
        ecc_stats["median"] = np.median(sel_ecc_vec)
        ecc_stats["range"] = (np.min(sel_ecc_vec), np.max(sel_ecc_vec))
        print_binned_stats(ecc_stats, 'Eccentricity')

    print()

def print_ecc_area_statistics(eccentricity_sel, area_all, ecc_binned_stats):
    """ Prints eccentricity and area statistics.  Assume that ecentricity and 
        area arrays are synchronized with their object IDs
    """

    total_obj_count = len(area_all)
    # Print binned statistics for eccentricity if provided
    for bin_range, ecc_stats in ecc_binned_stats.items():
        print(f"\t {bin_range}:")
        print(f"\t\t Count: {ecc_stats['count']}")
        print_binned_stats(ecc_stats, 'Eccentricity')
        #
        sel_indices = ecc_stats["item_indices"]
        sel_area_vec = area_all[sel_indices]
        area_stats = {}
        area_stats["count"] = len(sel_area_vec)
        area_stats["proportion"] = area_stats["count"] / total_obj_count
        area_stats["median"] = np.median(sel_area_vec)
        area_stats["range"] = (np.min(sel_area_vec), np.max(sel_area_vec))
        print_binned_stats(area_stats, 'Area')

    print()
    
def print_basic_stats(axon_measure, measure_label):
    
    # Calculate overall statistics
    count_measure = len(axon_measure)
    median_measure = np.median(axon_measure)
    min_measure = np.min(axon_measure)
    max_measure = np.max(axon_measure)

    # Print overall statistics for eccentricity
    print(f"\t Object {measure_label} Statistics (Overall):")
    print(f"\t\t Count: {count_measure}")
    print(f"\t\t Median: {median_measure:.2f}")
    print(f"\t\t Range: {min_measure:.2f} - {max_measure:.2f}\n")

def print_binned_stats(stats, measure_label):

    print(f"\t\t Proportion ({measure_label}): {stats['proportion']:.2%}")  # Format as percentage
    print(f"\t\t Median ({measure_label}): {stats['median']:.2f}")
    print(f"\t\t Range ({measure_label}): {stats['range'][0]:.2f} - {stats['range'][1]:.2f}")


Phase 3 Training

In [23]:
dataset_name = "Phase3_Training"  # Replace with your dataset name
#
image_root_folder = os.path.join("./datasets", dataset_name)
json_file = os.path.join(image_root_folder, f"{dataset_name}.json")

analyze_object_attributes(dataset_name, json_file, image_root_folder)


 Dataset: Phase3_Training; no of images: 30

 Basic statistics: Overall
	 Object Eccentricity Statistics (Overall):
		 Count: 4148
		 Median: 28.00
		 Range: 12.00 - 140.00

	 Object Area Statistics (Overall):
		 Count: 4148
		 Median: 1680.00
		 Range: 24.00 - 35288.00


 Basic stats: By category
	 Object Cat: 0 eccentricity Statistics (Overall):
		 Count: 2773
		 Median: 32.00
		 Range: 12.00 - 140.00

	 Object Cat: 0 area Statistics (Overall):
		 Count: 2773
		 Median: 2232.00
		 Range: 24.00 - 35288.00

	 Object Cat: 1 eccentricity Statistics (Overall):
		 Count: 1375
		 Median: 20.00
		 Range: 12.00 - 64.00

	 Object Cat: 1 area Statistics (Overall):
		 Count: 1375
		 Median: 1088.00
		 Range: 221.76 - 4304.00


 Statistics based on genric area binning: Overall
	 [0-1304):
		 Count: 1364
		 Proportion (Area): 32.88%
		 Median (Area): 960.00
		 Range (Area): 24.00 - 1302.96
		 Proportion (Eccentricity): 32.88%
		 Median (Eccentricity): 20.00
		 Range (Eccentricity): 12.00 - 60.00


Phase 3 Validation

In [24]:
dataset_name = "Phase3_Validation"  # Replace with your dataset name
#
image_root_folder = os.path.join("./datasets", dataset_name)
json_file = os.path.join(image_root_folder, f"{dataset_name}.json")

analyze_object_attributes(dataset_name, json_file, image_root_folder)


 Dataset: Phase3_Validation; no of images: 14

 Basic statistics: Overall
	 Object Eccentricity Statistics (Overall):
		 Count: 2793
		 Median: 28.00
		 Range: 12.00 - 176.00

	 Object Area Statistics (Overall):
		 Count: 2793
		 Median: 1712.00
		 Range: 171.52 - 19744.64


 Basic stats: By category
	 Object Cat: 0 eccentricity Statistics (Overall):
		 Count: 1836
		 Median: 32.00
		 Range: 12.00 - 176.00

	 Object Cat: 0 area Statistics (Overall):
		 Count: 1836
		 Median: 2376.00
		 Range: 171.52 - 19744.64

	 Object Cat: 1 eccentricity Statistics (Overall):
		 Count: 957
		 Median: 20.00
		 Range: 12.00 - 52.00

	 Object Cat: 1 area Statistics (Overall):
		 Count: 957
		 Median: 1088.00
		 Range: 256.00 - 4728.00


 Statistics based on genric area binning: Overall
	 [0-1304):
		 Count: 983
		 Proportion (Area): 35.20%
		 Median (Area): 952.00
		 Range (Area): 171.52 - 1296.00
		 Proportion (Eccentricity): 35.20%
		 Median (Eccentricity): 20.00
		 Range (Eccentricity): 12.00 - 76.0

Phase 3 Testing

In [25]:
dataset_name = "Phase3_Testing"  # Replace with your dataset name
#
image_root_folder = os.path.join("./datasets", dataset_name)
json_file = os.path.join(image_root_folder, f"{dataset_name}.json")

analyze_object_attributes(dataset_name, json_file, image_root_folder)


 Dataset: Phase3_Testing; no of images: 13

 Basic statistics: Overall
	 Object Eccentricity Statistics (Overall):
		 Count: 2573
		 Median: 28.00
		 Range: 12.00 - 284.00

	 Object Area Statistics (Overall):
		 Count: 2573
		 Median: 1696.00
		 Range: 32.00 - 20856.00


 Basic stats: By category
	 Object Cat: 0 eccentricity Statistics (Overall):
		 Count: 1684
		 Median: 32.00
		 Range: 12.00 - 108.00

	 Object Cat: 0 area Statistics (Overall):
		 Count: 1684
		 Median: 2320.00
		 Range: 32.00 - 20856.00

	 Object Cat: 1 eccentricity Statistics (Overall):
		 Count: 889
		 Median: 20.00
		 Range: 12.00 - 284.00

	 Object Cat: 1 area Statistics (Overall):
		 Count: 889
		 Median: 1073.60
		 Range: 248.08 - 5452.80


 Statistics based on genric area binning: Overall
	 [0-1304):
		 Count: 888
		 Proportion (Area): 34.51%
		 Median (Area): 984.00
		 Range (Area): 32.00 - 1301.36
		 Proportion (Eccentricity): 34.51%
		 Median (Eccentricity): 20.00
		 Range (Eccentricity): 12.00 - 284.00
	 

JHU / Quigley dataset

In [26]:
dataset_name = "Quigley_Eval"  # Replace with your dataset name
#
image_root_folder = os.path.join("./datasets", dataset_name)
json_file = os.path.join(image_root_folder, f"{dataset_name}.json")

analyze_object_attributes(dataset_name, json_file, image_root_folder)


 Dataset: Quigley_Eval; no of images: 51

 Basic statistics: Overall
	 Object Eccentricity Statistics (Overall):
		 Count: 14671
		 Median: 26.86
		 Range: 10.07 - 154.44

	 Object Area Statistics (Overall):
		 Count: 14671
		 Median: 1764.06
		 Range: 11.27 - 25001.25


 Basic stats: By category
	 Object Cat: 0 eccentricity Statistics (Overall):
		 Count: 10278
		 Median: 30.22
		 Range: 10.07 - 154.44

	 Object Cat: 0 area Statistics (Overall):
		 Count: 10278
		 Median: 2186.76
		 Range: 11.27 - 25001.25

	 Object Cat: 1 eccentricity Statistics (Overall):
		 Count: 4393
		 Median: 23.50
		 Range: 10.07 - 57.08

	 Object Cat: 1 area Statistics (Overall):
		 Count: 4393
		 Median: 1200.47
		 Range: 11.27 - 3292.21


 Statistics based on genric area binning: Overall
	 [0-1304):
		 Count: 4082
		 Proportion (Area): 27.82%
		 Median (Area): 1025.75
		 Range (Area): 11.27 - 1302.53
		 Proportion (Eccentricity): 27.82%
		 Median (Eccentricity): 20.14
		 Range (Eccentricity): 10.07 - 53.72